## Day 1

### テンソル

In [5]:
import torch

# 配列から初期化できる
data = [[1, 2],[3, 4]]
x_data = torch.tensor(data)

print(type(x_data))
print(x_data)

<class 'torch.Tensor'>
tensor([[1, 2],
        [3, 4]])


In [6]:
# テンソルの属性
tensor = torch.rand(3,4)

print(f"Shape of tensor: {tensor.shape}")
print(f"Datatype of tensor: {tensor.dtype}")
print(f"Device tensor is stored on: {tensor.device}")

Shape of tensor: torch.Size([3, 4])
Datatype of tensor: torch.float32
Device tensor is stored on: cpu


In [7]:
# x_dataの属性を維持して新しいテンソルを生成
x_ones = torch.ones_like(x_data) 
print(f"Ones Tensor: \n {x_ones} \n")

Ones Tensor: 
 tensor([[1, 1],
        [1, 1]]) 



### テンソルの操作

In [8]:
import torch

# GPUが使用可能であれば、GPU上にテンソルを移動させる
if torch.cuda.is_available():
    tensor = tensor.to('cuda')

In [13]:
# numpy likeな操作
tensor = torch.tensor(
    [
        [1,2,3],
        [4,5,6],
        [7,8,9],
    ]
)
print('First row: ',tensor[0])
print('First column: ', tensor[:, 0])
print('Last column:', tensor[..., -1])
print('Last row: ', tensor[-1, ...])
tensor[:,1] = 0
print(tensor)

"""MEMO
...は、pythonの組み込み定数である、Ellipsisというもの。
print(Ellipsis)
print(...)
...をインデックスに指定した場合、途中の(複数の)次元を省略できる。
:を指定した場合の複数形と捉えれば良い。
参考: https://note.nkmk.me/python-numpy-ellipsis/



-1をインデックスに指定した場合、「最後の要素」を表すインデックスとして利用されている。

import numpy as np
a = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9])
print(a[-1 : 100])# -1は最後の一つを表す
print(a[-1])


参考: https://blog.logicky.com/2017/01/25/python3-numpyとpythonの配列のスライスでマイナス使った場合/
""";

First row:  tensor([1, 2, 3])
First column:  tensor([1, 4, 7])
Last column: tensor([3, 6, 9])
Last row:  tensor([7, 8, 9])
tensor([[1, 0, 3],
        [4, 0, 6],
        [7, 0, 9]])


### データセットの読み込み

In [14]:
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)
"""Torch Visionとは
torchvision:
    公式Doc: https://pytorch.org/vision/stable/index.html
    画像タスクに対するデータセット、モデル、後処理、前処理等のスクリプトがまとまっている。
    
torchvision.datasets:
    多くのデータセットが利用できる
    ここにあるだけでもかなりのことができる。

mnist: 
    本家: http://yann.lecun.com/exdb/mnist/
    有名なデータセット集

LSUN:
    本家: https://www.yf.io/p/lsun
    室内風景のデータセット
    
ImageFolder:
    フォルダ分けされたデータセットを扱うためのDataset、Dataloaderが実装
    されている。
""";


"""DatasetとDataloader
Dataset:
    データを扱うためのpytorchのクラス。
    このクラスを拡張してDatasetクラスを作成する必要がある。
必須要件:
    __init__
    初期化処理
    
    __getitem__
    データセットのdata, labelを返す関数
    
    __len__:
    データセットの長さを返す関数

Dataloader:
    Datasetからデータを取り出すためのクラス
    dataloader.__iter__()により、iterに変換した上で、
    for文でループを回せば良い。
""";

In [38]:
# Datsetクラスから直接取り出す
## 最初の要素をタプルとして取得
data, label = (training_data.__getitem__(0))

print(f"label: {label}")
print(f"data.shape: {data.shape}")

## 配列のような取得もできる
# __getitem__は、Pythonの特殊メゾッドなので、以下のようにできる。
data, label = (training_data[0])
print(f"label: {label}")
print(f"data.shape: {data.shape}")

# dataloaderで中身を取り出す
train_loader = DataLoader(training_data, batch_size=10, shuffle=False)

for batch in iter(train_loader):
    # batchサイズの個数ずつ取り出される。
    print(f"len(batch): {len(batch)}")
    
    data, label = batch[0][0], batch[1][0]
    print(f"label: {label}")
    print(f"data.shape: {data.shape}") 
    break



label: 9
data.shape: torch.Size([1, 28, 28])
label: 9
data.shape: torch.Size([1, 28, 28])
len(batch): 2
label: 9
data.shape: torch.Size([1, 28, 28])


## データの加工

In [54]:
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda
ds = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
    target_transform=Lambda(lambda y: torch.zeros(10, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))
)

"""Transform
Transforms:
    データの変換を扱うクラス
    Datasetクラスの引数に渡すと、データを読み込むタイミングで変換を実行することができる。
    データを変換する際には、transform, 
    ラベルを変換するためには、target_transformの引数に設定する。

Lambda Transforms
    ユーザーが用意した関数を変換に利用できる仕組み。
    上の例では、スカラーをワンホット行列に変換する処理を指定している。
    y = torch.tensor(8)
    res = torch.zeros(10, dtype=torch.float).scatter_(0, torch.tensor(y), value=1)
    print(res)
    >>> tensor([0., 0., 0., 0., 0., 0., 0., 0., 1., 0.])
""";

### モデルの構築

In [3]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# 処理を実行するデバイスを選択
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cpu device


In [7]:
# モデルの実装
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(in_features=28*28, 
                      out_features=416),
            nn.ReLU(),
            nn.Linear(in_features=416, 
                      out_features=416),
            nn.ReLU(),
            nn.Linear(in_features=416, 
                      out_features=10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    
"""nn.Module
nn.Module:
    ニューラルネットワークを構築するためのPytorchのクラス。
    モデルを構築する場合には、このクラスを継承して中身を実装する。
    最低限、コンストラクタとforward関数を実装する必要がある。

forward:
    推論時の処理をここに実装する。

nn.Linear:
    全結合層を扱うためのクラス
nn.ReLU:
    活性化関数として有名な、「正規化線形関数」を扱うためのクラス。
    活性化関数は線形変換のあとに、非線形性を加え、ニューラルネットワークの表現力を向上させる役割をします
nn.Flatten
    高次元のデータを低次元に加工するレイヤー。
""";

In [8]:
# モデルの構造を確認
model = NeuralNetwork().to(device)
print(model)

"""
NeuralNetwork(
(flatten): Flatten(start_dim=1, end_dim=-1)
(linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
)
)
"""

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=416, bias=True)
    (1): ReLU()
    (2): Linear(in_features=416, out_features=416, bias=True)
    (3): ReLU()
    (4): Linear(in_features=416, out_features=10, bias=True)
    (5): ReLU()
  )
)


'\nNeuralNetwork(\n(flatten): Flatten(start_dim=1, end_dim=-1)\n(linear_relu_stack): Sequential(\n    (0): Linear(in_features=784, out_features=512, bias=True)\n    (1): ReLU()\n    (2): Linear(in_features=512, out_features=512, bias=True)\n    (3): ReLU()\n    (4): Linear(in_features=512, out_features=10, bias=True)\n    (5): ReLU()\n)\n)\n'

In [9]:
# 推論の実行
model = NeuralNetwork().to(device)
# 乱数で入力画像を作成
X = torch.rand(1, 28, 28, device=device)
logits = model(X) 
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([8])


In [23]:
# Flatten
input_image = torch.rand(3,28,28)
print(input_image.size())

flatten = nn.Flatten(
    start_dim=0,
    end_dim=1,
)
flat_image = flatten(input_image)
print(flat_image.size())

"""nn.Flatten
nn.Flatten
高次元のデータを一次元のデータに変換する。
start_dimからend_dimまでの次元のデータを1次元のデータに加工する。
ex): start_dim=0, end_dim=2,
     torch.Size([3, 28, 28])
     ↓
     torch.Size([2352])

ex): start_dim=1, end_dim=2,
     torch.Size([3, 28, 28])
     ↓
     torch.Size([3, 784])

ex): start_dim=0, end_dim=1,
     torch.Size([3, 28, 28])
     ↓
     torch.Size([84, 28])
""";

torch.Size([3, 28, 28])
torch.Size([84, 28])


In [30]:
# Flattenをより詳細に確かめる
input_image = torch.tensor(
    [
        [[0, 1, 2],
        [3, 4, 5],
        [6, 7, 8],],
        [[0, 1, 2],
        [3, 4, 5],
        [6, 7, 8],]
    ]
)
flatten = nn.Flatten(
    start_dim=1,
    end_dim=2,
)
flat_image = flatten(input_image)
print(input_image.size())
print(flat_image.size())
print(flat_image)

torch.Size([2, 3, 3])
torch.Size([2, 9])
tensor([[0, 1, 2, 3, 4, 5, 6, 7, 8],
        [0, 1, 2, 3, 4, 5, 6, 7, 8]])


In [27]:
flat_image

tensor([[0, 1, 2],
        [3, 4, 5],
        [6, 7, 8],
        [0, 1, 2],
        [3, 4, 5],
        [6, 7, 8]])